In [1]:
from fastlane_bot.bot import CarbonBot as Bot
from fastlane_bot.tools.cpc import ConstantProductCurve as CPC, CPCContainer, T, Pair
from fastlane_bot.tools.analyzer import CPCAnalyzer
from fastlane_bot.tools.cryptocompare import CryptoCompare
import requests
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(Bot))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CPC))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CPCAnalyzer))
import pandas as pd
import datetime
import time
import json
from hashlib import md5
from fastlane_bot import __VERSION__

Using default database url, if you want to use a different database, set the backend_url found at the bottom of manager_base.py
CarbonBot v3-b2.1 (03/May/2023)
ConstantProductCurve v2.13 (15/May/2023)
CPCAnalyzer v1.4 (13/May/2023)


# Mainnet Arbitrage Monitoring Bot [A015]

In [2]:
cid = lambda pair: md5(pair.encode()).hexdigest()
cid("WETH-6Cc2/USDC-eB48")

'ead90114986e463b0157c49422d8d465'

In [3]:
bot     = Bot()
CCm     = bot.get_curves()
fn = f"../data/A014-{int(time.time())}.csv.gz"
print (f"Saving as {fn}")
CCm.asdf().to_csv(fn, compression = "gzip")

Saving as ../data/A014-1684269508.csv.gz


In [4]:
class TokenAddress():
    def __init__(self, db):
        self._db = db
        
    def addr_from_ticker(self, ticker):
        return self._db.get_token(key=ticker).address
    a = addr_from_ticker
    
    def ticker_from_addr(self, addr):
        raise NotImplemented()
TA = TokenAddress(bot.db)      
TA.a("WETH-6Cc2")

'0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'

## Header and metadata

In [5]:
now = datetime.datetime.now()
print("*"*100)
print(f"ARBITRAGE ANALYSIS RUN @ {now.isoformat().split('.')[0]}Z [{int(now.timestamp())}]")
print("*"*100)

****************************************************************************************************
ARBITRAGE ANALYSIS RUN @ 2023-05-16T21:38:28Z [1684269508]
****************************************************************************************************


## Read curves

### Read Carbon curves

In [6]:
#CCm     = CPCContainer.from_df(pd.read_csv("../data/A014-1684148163.csv.gz"))
# CCu3    = CCm.byparams(exchange="uniswap_v3")
# CCu2    = CCm.byparams(exchange="uniswap_v2")
# CCs2    = CCm.byparams(exchange="sushiswap_v2")
CCc1    = CCm.byparams(exchange="carbon_v1")
# tc_u3   = CCu3.token_count(asdict=True)
# tc_u2   = CCu2.token_count(asdict=True)
# tc_s2   = CCs2.token_count(asdict=True)
# tc_c1   = CCc1.token_count(asdict=True)
# CAm     = CPCAnalyzer(CCm)
CAc1 = CPCAnalyzer(CCc1)
pairs  = CAc1.pairsc()

now = datetime.datetime.now()
int(now.timestamp()), now.isoformat()## Print heading

### Read prices and create proxy curves

In [7]:
tokens0 = CAc1.tokens()
tokens0

{'0x0-1AD5',
 'ARB-4ad1',
 'BNT-FF1C',
 'CRV-cd52',
 'DAI-1d0F',
 'ETH2x_FLI-65BD',
 'LBR-aCcA',
 'LINK-86CA',
 'LYXe-be6D',
 'MATIC-eBB0',
 'PEPE-1933',
 'RPL-A51f',
 'SMT-7173',
 'TSUKA-69eD',
 'USDC-eB48',
 'USDT-1ec7',
 'WBTC-C599',
 'WETH-6Cc2',
 'XCHF-fc08',
 'eRSDL-D3A6',
 'stETH-fE84',
 'vBNT-7f94'}

In [8]:
print("\n\n"+"="*100)
print("REMOVED TOKENS")
print("="*100)



REMOVED TOKENS


In [9]:
REMOVED_TOKENS = {"0x0-1AD5", "LBR-aCcA"}
print(REMOVED_TOKENS)

{'0x0-1AD5', 'LBR-aCcA'}


In [10]:
tokens = tokens0 - REMOVED_TOKENS
pairs  = CAc1.CC.filter_pairs(bothin=tokens)
tokens_addr = {tkn: TA.a(tkn) for tkn in tokens}
tokens_addrr = {v.lower():k for k,v in tokens_addr.items()}
print("\n\n"+"="*100)
print("TOKEN ADDRESSES")
print("="*100)
for k,v in tokens_addr.items():
    print(f"{k:20} {v}")
tokens_addr, tokens_addrr



TOKEN ADDRESSES
WETH-6Cc2            0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
BNT-FF1C             0x1F573D6Fb3F13d689FF844B4cE37794d79a7FF1C
PEPE-1933            0x6982508145454Ce325dDbE47a25d4ec3d2311933
eRSDL-D3A6           0x5218E472cFCFE0b64A064F055B43b4cdC9EfD3A6
USDC-eB48            0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48
LINK-86CA            0x514910771AF9Ca656af840dff83E8264EcF986CA
CRV-cd52             0xD533a949740bb3306d119CC777fa900bA034cd52
LYXe-be6D            0xA8b919680258d369114910511cc87595aec0be6D
SMT-7173             0xB17548c7B510427baAc4e267BEa62e800b247173
stETH-fE84           0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84
WBTC-C599            0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599
RPL-A51f             0xD33526068D116cE69F19A9ee46F0bd304F21A51f
MATIC-eBB0           0x7D1AfA7B718fb893dB30A3aBc0Cfc608AaCfeBB0
XCHF-fc08            0xB4272071eCAdd69d933AdcD19cA99fe80664fc08
USDT-1ec7            0xdAC17F958D2ee523a2206206994597C13D831ec7
ARB-4ad1             0

({'WETH-6Cc2': '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2',
  'BNT-FF1C': '0x1F573D6Fb3F13d689FF844B4cE37794d79a7FF1C',
  'PEPE-1933': '0x6982508145454Ce325dDbE47a25d4ec3d2311933',
  'eRSDL-D3A6': '0x5218E472cFCFE0b64A064F055B43b4cdC9EfD3A6',
  'USDC-eB48': '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48',
  'LINK-86CA': '0x514910771AF9Ca656af840dff83E8264EcF986CA',
  'CRV-cd52': '0xD533a949740bb3306d119CC777fa900bA034cd52',
  'LYXe-be6D': '0xA8b919680258d369114910511cc87595aec0be6D',
  'SMT-7173': '0xB17548c7B510427baAc4e267BEa62e800b247173',
  'stETH-fE84': '0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84',
  'WBTC-C599': '0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599',
  'RPL-A51f': '0xD33526068D116cE69F19A9ee46F0bd304F21A51f',
  'MATIC-eBB0': '0x7D1AfA7B718fb893dB30A3aBc0Cfc608AaCfeBB0',
  'XCHF-fc08': '0xB4272071eCAdd69d933AdcD19cA99fe80664fc08',
  'USDT-1ec7': '0xdAC17F958D2ee523a2206206994597C13D831ec7',
  'ARB-4ad1': '0xB50721BCf8d664c30412Cfbc6cf7a15145234ad1',
  'ETH2x_FLI-65BD': '0xAa6

In [11]:
addr_s = ",".join(x for x in tokens_addr.values())
url = "https://api.coingecko.com/api/v3/simple/token_price/ethereum"
params = dict(contract_addresses=addr_s, vs_currencies="usd")
r = requests.get(url, params=params)
token_prices_usd_cg_raw = {tokens_addrr[k]: v["usd"] for k,v in r.json().items()}
token_prices_usd_cg = {Pair.n(tokens_addrr[k]).upper(): v["usd"] for k,v in r.json().items()}
token_prices_usd_cg_raw

{'eRSDL-D3A6': 0.00246784,
 'LYXe-be6D': 13.35,
 'CRV-cd52': 0.826124,
 'RPL-A51f': 49.63,
 'USDT-1ec7': 1.0,
 'PEPE-1933': 1.68e-06,
 'BNT-FF1C': 0.417567,
 'WETH-6Cc2': 1819.77,
 'USDC-eB48': 1.0,
 'stETH-fE84': 1816.58,
 'ARB-4ad1': 1.17,
 'TSUKA-69eD': 0.03481537,
 'SMT-7173': 0.065828,
 'XCHF-fc08': 1.11,
 'LINK-86CA': 6.64,
 'MATIC-eBB0': 0.841915,
 'vBNT-7f94': 0.371528,
 'ETH2x_FLI-65BD': 11.91,
 'DAI-1d0F': 1.001,
 'WBTC-C599': 27005}

In [12]:
missing_cg = set(tokens_addr) - set(token_prices_usd_cg_raw)
missing_cg

set()

In [13]:
tokens_cc = [Pair.n(x) for x in tokens]
tokens_cc

['WETH',
 'BNT',
 'PEPE',
 'eRSDL',
 'USDC',
 'LINK',
 'CRV',
 'LYXe',
 'SMT',
 'stETH',
 'WBTC',
 'RPL',
 'MATIC',
 'XCHF',
 'USDT',
 'ARB',
 'ETH2x_FLI',
 'TSUKA',
 'vBNT',
 'DAI']

In [14]:
# token_prices_usd_cc = CryptoCompare(apikey=True).query_tokens(tokens_cc)
# token_prices_usd_cc

In [15]:
# missing_cc = set(tokens_cc) - set(token_prices_usd_cc)
# missing_cc

In [16]:
token_prices_usd = token_prices_usd_cg

In [17]:
P0 = lambda tknb,tknq: token_prices_usd[tknb.upper()]/token_prices_usd[tknq.upper()]
def P(pair):
    try: 
        return P0(*Pair.n(pair).split("/"))
    except KeyError:
        return None
P("WBTC-xxx/WETH-xxx")

14.83978744566621

In [18]:
prices_by_pair = {pair: P(pair) for pair in pairs}
prices_n_by_pair = {Pair.n(pair): p for pair, p in prices_by_pair.items()}
print("\n\n"+"="*100)
print("PRICES BY PAIR")
print("="*100)
for k,v in prices_n_by_pair.items():
    if not v is None:
        print(f"{k:20} {v:20,.6f}")
    else:
        print(f"{k:20}                  ---")



PRICES BY PAIR
SMT/WETH                         0.000036
PEPE/WETH                        0.000000
WETH/USDT                    1,819.770000
BNT/WETH                         0.000229
eRSDL/WETH                       0.000001
WBTC/USDT                   27,005.000000
ARB/MATIC                        1.389689
LYXe/USDC                       13.350000
WBTC/USDC                   27,005.000000
WBTC/WETH                       14.839787
vBNT/USDC                        0.371528
CRV/USDC                         0.826124
BNT/vBNT                         1.123918
USDT/USDC                        1.000000
LINK/USDT                        6.640000
RPL/XCHF                        44.711712
stETH/WETH                       0.998247
BNT/USDC                         0.417567
WETH/DAI                     1,817.952048
LINK/USDC                        6.640000
TSUKA/USDC                       0.034815
ETH2x_FLI/WETH                   0.006545
DAI/USDT                         1.001000
WETH/USDC        

In [19]:
proxy_curves = [
    CPC.from_pk(p=price, pair=pair, k=1000, cid=cid(pair), params=dict(exchange="coingecko")) 
    for pair, price in prices_by_pair.items() if not price is None
]
#proxy_curves

In [20]:
# CCother = CCu3.bypairs(CCc1.pairs())
CCother = CPCContainer(proxy_curves)
CCfull = CCc1.copy().add(CCother)
CAother = CPCAnalyzer(CCother)
CAfull = CPCAnalyzer(CCfull)

## High level reference data

In [21]:
print("\n\n                    CARBON    REFERENCE")
print(f"Pairs:                {len(pairs):4}    [{len(CCother.pairs()):7}]")
print(f"Tokens:               {len(CAc1.tokens()):4}    [{len(CAother.tokens()):7}]")
print(f"Curves:               {len(CAc1.CC):4}    [{len(CAother.CC):7}]")



                    CARBON    REFERENCE
Pairs:                  25    [     25]
Tokens:                 22    [     20]
Curves:                 78    [     25]


## By pair data for Carbon

### Count by pairs

In [22]:
df = CAc1.count_by_pairs()
print("\n\n"+"="*100)
print("PAIRS")
print("="*100)
print(df)
#df



PAIRS
                          count
pair                           
WETH-6Cc2/USDC-eB48          17
BNT-FF1C/WETH-6Cc2            9
BNT-FF1C/vBNT-7f94            9
USDT-1ec7/USDC-eB48           5
stETH-fE84/WETH-6Cc2          4
WBTC-C599/WETH-6Cc2           3
DAI-1d0F/USDC-eB48            3
WBTC-C599/USDC-eB48           2
DAI-1d0F/USDT-1ec7            2
PEPE-1933/WETH-6Cc2           2
ARB-4ad1/MATIC-eBB0           2
LINK-86CA/USDT-1ec7           2
CRV-cd52/USDC-eB48            2
WETH-6Cc2/USDT-1ec7           2
0x0-1AD5/WETH-6Cc2            2
BNT-FF1C/USDC-eB48            1
LINK-86CA/USDC-eB48           1
WBTC-C599/USDT-1ec7           1
LBR-aCcA/WETH-6Cc2            1
RPL-A51f/XCHF-fc08            1
TSUKA-69eD/USDC-eB48          1
LYXe-be6D/USDC-eB48           1
SMT-7173/WETH-6Cc2            1
eRSDL-D3A6/WETH-6Cc2          1
ETH2x_FLI-65BD/WETH-6Cc2      1
WETH-6Cc2/DAI-1d0F            1
vBNT-7f94/USDC-eB48           1


### Calculate by-pair statistics

In [23]:
pasdf = CAfull.pool_arbitrage_statistics()

### Print by-pair statistics

In [36]:
for pair in list(pairs):
    print("\n\n"+"="*100)
    print(f"Pair = {pair}")
    print("="*100)
    df = pasdf.loc[Pair.n(pair)]
    hasproxydata = len(df.reset_index()[df.reset_index()["exchange"]=="coingecko"])>0
    if hasproxydata:
        print("\n--- ALL POSITIONS ---")
        print(df.to_string())
        print("\n--- IN-THE-MONEY POSITIONS ---")
        dfitm = df[df["itm"]=="x"]
        if len(dfitm) > 0:
            print(dfitm.to_string())
        else:
            print("-None-")
    else:
        print("\n--- NO PRICE DATA AVAILABLE ---")



Pair = SMT-7173/WETH-6Cc2

--- ALL POSITIONS ---
                       price             vl itm  b  s                               bsv
exchange  cid0                                                                         
coingecko 26a0228a  0.000036   10515.572355      b  s  buy-sell-SMT @ 0.00 WETH per SMT
carbon_v1 343738-1  0.000080  200000.000000         s      sell-SMT @ 0.00 WETH per SMT

--- IN-THE-MONEY POSITIONS ---
-None-


Pair = PEPE-1933/WETH-6Cc2

--- ALL POSITIONS ---
                           price            vl itm  b  s                                 bsv
exchange  cid0                                                                              
coingecko 394dbcea  9.231936e-10  2.081534e+06      b  s  buy-sell-PEPE @ 0.00 WETH per PEPE
carbon_v1 440620-1  4.000000e-07  7.144675e+06         s      sell-PEPE @ 0.00 WETH per PEPE
          440621-1  4.500000e-07  1.315789e+06         s      sell-PEPE @ 0.00 WETH per PEPE

--- IN-THE-MONEY POSITIONS ---
-None-




## Summary data

### Create summary data

In [25]:
itmcarbdf = pasdf.query("exchange == 'carbon_v1'").query("itm == 'x'")

In [26]:
itmcarb_pairs = sorted({x[0] for x in tuple(itmcarbdf.index)})
itmcarb_pairs

['ARB/MATIC', 'BNT/WETH', 'BNT/vBNT', 'CRV/USDC', 'LINK/USDT', 'WETH/USDT']

In [27]:
itmcarb_pos = itmcarbdf.reset_index().to_dict(orient="records")
itmcarb_pos[:2]

[{'pair': 'ARB/MATIC',
  'exchange': 'carbon_v1',
  'cid0': '806240-1',
  'price': 1.4285714285714268,
  'vl': 141.80602335295742,
  'itm': 'x',
  'b': 'b',
  's': '',
  'bsv': 'buy-ARB @ 1.43 MATIC per ARB'},
 {'pair': 'BNT/WETH',
  'exchange': 'carbon_v1',
  'cid0': '326076-0',
  'price': 0.0002401849418126649,
  'vl': 12.002104931254514,
  'itm': 'x',
  'b': 'b',
  's': '',
  'bsv': 'buy-BNT @ 0.00 WETH per BNT'}]

In [28]:
itmcarb_pos_bypair = {
    pair: [x for x in itmcarb_pos if x["pair"] == pair]
    for pair in itmcarb_pairs
}
#itmcarb_pos_bypair

In [29]:
missing_pairs = [pair for pair, price in prices_by_pair.items() if price is None]
missing_pairs

[]

### Convert summary data to Telegram

In [30]:
telegram_data = dict(
    time_ts = int(now.timestamp()),              # timestamp (epoch)
    time_iso = now.isoformat().split('.')[0],    # timestap (iso format)
    prices_usd = token_prices_usd,               # token prices (usd)
    pairs = list(pairs),                         # all pairs
    pairs_n = len(pairs),                        # ...number
    itm_pairs = itmcarb_pairs,                   # pairs that have curves in the money (list)
    itm_pairs_n = len(itmcarb_pairs),            # ...number
    itm_pos = itmcarb_pos,                       # carbon positions that are in the money (list)
    itm_pos_n = len(itmcarb_pos),                # ...number
    itm_pos_bypair = itmcarb_pos_bypair,         # ditto, but dict[pair] -> list
    missing_pairs = missing_pairs,               # missing pairs
    missing_pairs_n = len(missing_pairs),        # ...number
    removed_tokens = list(REMOVED_TOKENS),       # removed tokens
    removed_tokens_n = len(REMOVED_TOKENS),      # ...number
)

In [31]:
td = telegram_data
s = ""
s += f"="*47
s += f"\nARBITRAGE RUN @ {td['time_iso']}Z\n"
s += f"="*47+"\n"
s += f"Removed tokens:          {td['removed_tokens_n']:3}\n"
s += f"Total pairs:             {td['pairs_n']:3}\n"
s += f"Missing pairs:           {td['missing_pairs_n']:3}\n"
s += f"In-the-money pairs:      {td['itm_pairs_n']:3}\n"
s += f"In-the-money curves:     {td['itm_pos_n']:3}\n"
total_vl_usd = 0
total_arbval = 0
s += "-----------------------------------------------\n"
s += "PAIR         CID          VLOCK    ARBPC    VAL\n"
s += "-----------------------------------------------\n"
for p in td['itm_pos']:
    price_pair = prices_n_by_pair[p['pair']] or 0
    price_pc = f"{abs(price_pair/p['price']-1)*100:8.1f}%"
    vl_token = p['pair'].split('/')[0].split("-")[0]
    vl_token_price = token_prices_usd.get(vl_token.upper())
    vl_usd = p['vl']*vl_token_price
    total_vl_usd += vl_usd
    arbval = vl_usd * abs(price_pair/p['price']-1)
    if price_pc.endswith("100.0%"): 
        price_pc = "         "
        arbval = 0
    total_arbval += arbval
    s += f"{p['pair']:12} "
    s += f"{p['cid0'][-8:]:8} "
    s += f"{vl_usd:9,.0f}"
    s += f"{price_pc} "
    s += f"{arbval:6,.0f}"
    #s += f"[{p['bsv']}; p={price_pair:,.2f}]"
    #s += f"\n{p}"
    s += "\n"
s += "-----------------------------------------------\n"
s += f"TOTAL {total_vl_usd:25,.0f}   {100*total_arbval/total_vl_usd:5.1f}% {total_arbval:6,.0F}\n"
s += "===============================================\n"
s += "\n\n"
telegram_data["text"] = s
print()
print(s)


ARBITRAGE RUN @ 2023-05-16T21:38:28Z
Removed tokens:            2
Total pairs:              25
Missing pairs:             0
In-the-money pairs:        6
In-the-money curves:       7
-----------------------------------------------
PAIR         CID          VLOCK    ARBPC    VAL
-----------------------------------------------
ARB/MATIC    806240-1       166     2.7%      5
BNT/WETH     326076-0         5     4.5%      0
BNT/vBNT     748990-0         0     1.2%      0
BNT/vBNT     748977-1       167    10.1%     17
CRV/USDC     612490-0         1     8.2%      0
LINK/USDT    960408-0         0     3.8%      0
WETH/USDT    691656-0         5     3.8%      0
-----------------------------------------------
TOTAL                       345     6.4%     22





In [32]:
with open("Analysis_015.latest.out", "w") as f:
    f.write(s)

In [33]:
with open("Analysis_015.latest.json", "w") as f:
    f.write(json.dumps(telegram_data))

In [34]:
None or 0

0

## Review

In [35]:
#print(CCfull.bycids(endswith="612490-0")[0].description())